In [2]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests

In [5]:

with open('Ata/unpacked/Crops.json', 'r') as f:
    data = json.load(f)
f.close()
with open('Ata/unpacked/ObjectInformation.json', 'r') as f:
    data_info = json.load(f)
f.close()
info = pd.DataFrame(data_info['content'],index=[0])
info = info.T
crops = pd.DataFrame(data['content'],index=[0])
crops=crops.T
crops['Name'] = None
crops['Buy'] = None
crops['Sell Normal'] = None
crops['Sell Silver'] = None
crops['Sell Gold'] = None
crops['Growth'] = None
crops['Season'] = None
crops['Regrowth'] = None
crops['Harvest Method'] = None
crops['Extra Harvest'] = None
crops[0]=crops[0].apply(lambda x: x.split('/'))
idxs = list(crops.index.values)
for idx in idxs:
    crops.loc[idx,'Name'] = info.loc[idx,0].split('/')[0]
    crops.loc[idx,'Growth']= crops.loc[idx,0][0]
    crops.loc[idx,'Season']= crops.loc[idx,0][1]
    crops.loc[idx,'Regrowth']= crops.loc[idx,0][4]
    crops.loc[idx,'Harvest Method']= crops.loc[idx,0][5]
    crops.loc[idx,'Extra Harvest']= crops.loc[idx,0][6]
crops = crops.drop(0,axis = 1)
crops['Harvest'] = crops['Harvest'].apply(lambda x: int(x))
crops['Harvest Method'] = crops['Harvest Method'].apply(lambda x: int(x))
crops['Regrowth'] = crops['Regrowth'].apply(lambda x: int(x))
crops['Harvest Mode'] = crops['Harvest Method'].apply(lambda x: 'Scythe' if x== 1 else 'Normal' )
crops['Regrowth'] = crops['Regrowth'].apply(lambda x: None if x == -1 else x)
crops['Growth'] = crops['Growth'].apply(lambda x: convert(x))
crops['Regrowth'] = crops['Regrowth'].apply(lambda x: int(x) if x/1==x  else 0)
crops.to_csv('Data/data.csv')
crops = crops.drop(['495','496','497','498'],axis=0)

crops

,Name,Buy,Sell Normal,Sell Silver,Sell Gold,Growth,Season,Harvest,Regrowth,Harvest Method,Extra Harvest,Harvest Mode
299,Amaranth Seeds,None,None,None,None,7,fall,300,0,1,false,Scythe
301,Grape Starter,None,None,None,None,10,fall,398,3,0,true 1 2 6 0,Normal
302,Hops Starter,None,None,None,None,11,summer,304,1,0,true 1 2 6 0,Normal
347,Rare Seed,None,None,None,None,24,fall,417,0,0,false,Normal
425,Fairy Seeds,None,None,None,None,12,fall,595,0,0,false,Normal
427,Tulip Bulb,None,None,None,None,6,spring,591,0,0,false,Normal
429,Jazz Seeds,None,None,None,None,7,spring,597,0,0,false,Normal
431,Sunflower Seeds,None,None,None,None,8,summer fall,421,0,0,false,Normal
433,Coffee Bean,None,None,None,None,10,spring summer,433,2,0,true 4 6 10 .02,Normal
453,Poppy Seeds,None,None,None,None,7,summer,376,0,0,false,Normal


In [4]:
def convert(x):
    if x == 'false':
        return 0
    else:
        result = sum(list(map(int, x.split(' '))))
        return result
    
def takeg(x):
    return int(x[:-1])

In [6]:
def parse_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    tables = soup.find_all('table')
    return tables
def parse_html_table(table):
    df = pd.read_html(str(table)) 
    return df

for i in range(2,109):
    try:
        url = 'https://stardewvalleywiki.com/Crops'
        tables = parse_url(url)
        df = parse_html_table(tables[i])
        name,surname = df[0]['Seeds'][0].split(' ')[:2]
        name += ' '
        name += surname
        buy  = float(df[0]['Seeds'][0].split(' ')[4][:-1])
        sells = df[0].loc[0,'Sells For'].split(' ')
        sells = list(map(takeg,sells))

        crops.loc[crops['Name']==name,'Buy'] = buy
        crops.loc[crops['Name']==name,'Sell Normal'] = sells[0]
        crops.loc[crops['Name']==name,'Sell Silver'] = sells[1]
        crops.loc[crops['Name']==name,'Sell Gold'] = sells[2]    
        print(i)
    except (KeyError,ValueError)as e:
        pass
    



2
5
10
13
16
19
22
25
30
33
36
39
42
45
48
51
57
60
63
66
69
71
74
77
83
86
89
92
95
97
102


In [7]:
#Rare Seeds
crops.loc['347','Buy'] = 1000
crops.loc['347','Sell Normal'] = 3000
crops.loc['347','Sell Silver'] = 3750
crops.loc['347','Sell Gold'] = 4500  

#Coffee Bean
crops.loc['433','Buy'] = 2500
crops.loc['433','Sell Normal'] = 15
crops.loc['433','Sell Silver'] = 18
crops.loc['433','Sell Gold'] = 20  

#Red Cabbage Seeds
crops.loc['485','Buy'] = 100
crops.loc['485','Sell Normal'] = 260
crops.loc['485','Sell Silver'] = 325
crops.loc['485','Sell Gold'] = 390

#Bok Choy Seeds
crops.loc['491','Buy'] = 50
crops.loc['491','Sell Normal'] = 80
crops.loc['491','Sell Silver'] = 100
crops.loc['491','Sell Gold'] = 120

#Ancient Seeds
crops.loc['499','Buy'] = 550
crops.loc['499','Sell Normal'] = 550
crops.loc['499','Sell Silver'] = 687
crops.loc['499','Sell Gold'] = 825

#Strawberry Seeds
crops.loc['745','Buy'] = 100
crops.loc['745','Sell Normal'] = 120
crops.loc['745','Sell Silver'] = 150
crops.loc['745','Sell Gold'] = 180

In [24]:
crops = crops.dropna()
crops.iloc[2,]

Name              Hops Starter
Buy                         60
Sell Normal                 25
Sell Silver                 31
Sell Gold                   37
Growth                      11
Season                  summer
Harvest                    304
Regrowth                     1
Harvest Method               0
Extra Harvest     true 1 2 6 0
Harvest Mode            Normal
Name: 302, dtype: object

In [38]:
area = '7x15'
w = int(area.split('x')[0])
h = int(area.split('x')[1])
money = 1000
season = 'summer'
farmlvl = 3
l =[]
for i in range(len(crops)):
    if season in crops.iloc[i,6].split(' '):
        l.append(i)
l

[2, 7, 8, 9, 10, 18, 19, 20, 21, 22, 23, 24, 25, 26, 34, 36]

In [37]:
idx = 2
qtd  = int(money/crops.iloc[idx,1])
harvest_days= crops.iloc[idx,5]
harvs = 1
while harvest_days <= 28:
    harvest_days+=crops.iloc[idx,8]
    harvs+=1
harvest_days -= crops.iloc[idx,8]
harvs-=1
sell = crops.iloc[idx,2]
harvs*qtd*sell

7200